In [1]:
import pandas as pd

In [2]:
bat_df = pd.read_csv('../Datasets/batter_stats.csv')
pitch_df = pd.read_csv('../Datasets/pitcher_stats.csv')
lineups = pd.read_csv('../Datasets/Lineups_2021_06_28.csv')

In [3]:
# The dots in names like J.T. were inconsistent across datasets
bat_df.loc[:,'Name'] = bat_df['Name'].str.replace('.', '')
pitch_df.loc[:,'Name'] = pitch_df['Name'].str.replace('.', '')
lineups.loc[:,' player name'] = lineups[' player name'].str.replace('.', '')

In [4]:
bats_r = bat_df[bat_df['bats'] == 'R']
bats_l = bat_df[bat_df['bats'] == 'L']

In [5]:
# Making name column consistent
lineups.columns = lineups.columns.str.replace(' player name', 'Name')

In [6]:
# Individual name that was giving trouble
lineups.loc[lineups['Name'] == 'Jonathan Gray', 'Name'] = 'Jon Gray'

In [7]:
# Creates a list of teams with no repeats
# The order needs to be kept from the dataframe because 
# opposing teams are paired
team_list = []
for team in lineups['team code']:
    if team not in team_list:
        team_list.append(team)
# Creating a dictionary of every team, and the team they face
matchups = {}
for i in range(1, len(team_list), 2):
    matchups[team_list[i]] = team_list[i-1]
    matchups[team_list[i-1]] = team_list[i]

In [8]:
lineups

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position
0,PIT,6/28/2021,1,624428,Adam Frazier,1,N,NaN
1,PIT,6/28/2021,1,663647,Ke'Bryan Hayes,2,N,NaN
2,PIT,6/28/2021,1,668804,Bryan Reynolds,3,N,NaN
3,PIT,6/28/2021,1,607732,Jacob Stallings,4,N,NaN
4,PIT,6/28/2021,1,595943,Phillip Evans,5,N,NaN
...,...,...,...,...,...,...,...,...
203,LAD,6/28/2021,1,669257,Will Smith,5,N,NaN
204,LAD,6/28/2021,1,621035,Chris Taylor,6,N,NaN
205,LAD,6/28/2021,1,666158,Gavin Lux,7,N,NaN
206,LAD,6/28/2021,1,572041,AJ Pollock,8,N,NaN


In [9]:
batters = lineups.loc[(lineups[' batting order'] != 'SP') & (lineups[' game_number'] == 1)]

In [10]:
pitch_df.drop(456, inplace = True)

In [11]:
matchups

{'COL': 'PIT',
 'PIT': 'COL',
 'CIN': 'PHI',
 'PHI': 'CIN',
 'NYY': 'LAA',
 'LAA': 'NYY',
 'WAS': 'NYM',
 'NYM': 'WAS',
 'BOS': 'KC',
 'KC': 'BOS',
 'CLE': 'DET',
 'DET': 'CLE',
 'MIL': 'CHC',
 'CHC': 'MIL',
 'HOU': 'BAL',
 'BAL': 'HOU',
 'CHW': 'MIN',
 'MIN': 'CHW',
 'STL': 'ARI',
 'ARI': 'STL',
 'LAD': 'SF',
 'SF': 'LAD'}

In [12]:
opp_pitch_hand = []
opp_pitcher = []
del_indexes = []
keep_pitch = []
for i, row in batters.iterrows():
    if len(lineups.loc[(lineups['team code'] == matchups[row['team code']])\
                            & (lineups[' batting order'] == 'SP') & (lineups[' game_number'] == 1), 'Name'].values) > 0: 
        opp_pitch = lineups.loc[(lineups['team code'] == matchups[row['team code']])\
                                & (lineups[' batting order'] == 'SP') & (lineups[' game_number'] == 1), 'Name'].values[0]
        if len(pitch_df.sort_values(by = 'IP', ascending = False).loc[pitch_df['Name'] == opp_pitch, 'throws'].values) > 0:
                opp_pitch_hand.append(pitch_df.sort_values(by = 'IP', ascending = False).loc[pitch_df['Name'] == opp_pitch, 'throws'].values[0])
                opp_pitcher.append(opp_pitch)
                keep_pitch.append(opp_pitch)
        else:
            del_indexes.append(i)
    else:
        del_indexes.append(i)
batters.drop(index = del_indexes, inplace = True)
batters['opp_pitch_hand'] = opp_pitch_hand
batters['opp_pitch'] = opp_pitcher


/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-12-a4b56b56add5>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batters['opp_pitch_hand'] = opp_pitch_hand
<ipython-input-12-a4b56b56add5>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

In [13]:
bats = []
k_pct = []
bb_pct = []
bb_k = []
bat_avg = []
obp = []
slg = []
ops = []
iso = []
babip = []
wrc = []
wraa = []
woba = []
wrc_plus = []
for i, row in batters.iterrows():
    # Pulling stats based on handedness of pitcher
    if row['opp_pitch_hand'] == 'R':
        batter_df = bat_df.loc[(bat_df['pitch']== 'R')&(bat_df['Name']==row['Name'])]
        # Checking for switch hitters
        if len(batter_df) == 2:
            # A switch hitter will hit using the opposite hand of the pitcher
            batter_df = batter_df.loc[batter_df['bats']== 'L']
    if row['opp_pitch_hand'] == 'L':
        batter_df = bat_df.loc[(bat_df['pitch']== 'L')&(bat_df['Name']==row['Name'])]
        # Checking for switch hitters
        if len(batter_df) == 2:
            # A switch hitter will hit using the opposite hand of the pitcher
            batter_df = batter_df.loc[batter_df['bats']== 'R']
    if len(batter_df) == 0:
        for col in batter_df.columns:
            batter_df[col] = [float('NaN')]
    bats.append(batter_df['bats'].values[0])
    k_pct.append(batter_df['K%'].values[0])
    bb_pct.append(batter_df['BB%'].values[0])
    bb_k.append(batter_df['BB/K'].values[0])
    bat_avg.append(batter_df['AVG'].values[0])
    obp.append(batter_df['OBP'].values[0])
    slg.append(batter_df['SLG'].values[0])
    ops.append(batter_df['OPS'].values[0])
    iso.append(batter_df['ISO'].values[0])
    babip.append(batter_df['BABIP'].values[0])
    wrc.append(batter_df['wRC'].values[0])
    wraa.append(batter_df['wRAA'].values[0])
    woba.append(batter_df['wOBA'].values[0])
    wrc_plus.append(batter_df['wRC+'].values[0])
batters.loc[:,'bats'] = bats
batters.loc[:,'k_pct'] = k_pct
batters.loc[:,'bb_pct'] = bb_pct
batters.loc[:,'bb_k'] = bb_k
batters.loc[:,'bat_avg'] = bat_avg
batters.loc[:,'obp'] = obp
batters.loc[:,'slg'] = slg
batters.loc[:,'ops'] = ops
batters.loc[:,'iso'] = iso
batters.loc[:,'babip'] = babip
batters.loc[:,'wrc'] = wrc
batters.loc[:,'wraa'] = wraa
batters.loc[:,'woba'] = woba
batters.loc[:,'wrc_plus'] = wrc_plus

<ipython-input-13-0499cefeeb18>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batter_df[col] = [float('NaN')]
/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

In [14]:
batters

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,opp_pitch_hand,opp_pitch,...,bat_avg,obp,slg,ops,iso,babip,wrc,wraa,woba,wrc_plus
0,PIT,6/28/2021,1,624428,Adam Frazier,1,N,NaN,L,Kyle Freeland,...,0.263158,0.320388,0.400000,0.720388,0.136842,0.302632,12.389302,-0.048698,0.315698,98.911367
1,PIT,6/28/2021,1,663647,Ke'Bryan Hayes,2,N,NaN,L,Kyle Freeland,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PIT,6/28/2021,1,668804,Bryan Reynolds,3,N,NaN,L,Kyle Freeland,...,0.293478,0.375000,0.510870,0.885870,0.217391,0.403226,17.904202,5.349609,0.378643,140.803722
3,PIT,6/28/2021,1,607732,Jacob Stallings,4,N,NaN,L,Kyle Freeland,...,0.265625,0.389610,0.546875,0.936485,0.281250,0.302326,14.800571,5.456544,0.400681,154.733712
4,PIT,6/28/2021,1,595943,Phillip Evans,5,N,NaN,L,Kyle Freeland,...,0.236842,0.309524,0.342105,0.651629,0.105263,0.275862,4.369694,-0.652318,0.290388,83.287519
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,LAD,6/28/2021,1,641355,Cody Bellinger,4,N,NaN,R,Anthony DeSclafani,...,0.246753,0.344633,0.493506,0.838139,0.246753,0.245763,27.390634,5.323999,0.354311,125.907423
203,LAD,6/28/2021,1,669257,Will Smith,5,N,NaN,R,Anthony DeSclafani,...,0.308642,0.425000,0.604938,1.029938,0.296296,0.299213,43.343418,19.158044,0.431390,178.210642
204,LAD,6/28/2021,1,621035,Chris Taylor,6,N,NaN,R,Anthony DeSclafani,...,0.283333,0.392226,0.450000,0.842226,0.166667,0.372671,47.437224,12.948708,0.370080,137.743822
205,LAD,6/28/2021,1,666158,Gavin Lux,7,N,NaN,R,Anthony DeSclafani,...,0.234177,0.290698,0.398734,0.689432,0.164557,0.271930,18.268869,-2.303424,0.296797,90.348827


In [15]:
k_9 = []
bb_9 = []
k_bb = []
hr_9 = []
k_pct = []
bb_pct = []
k_bb_pct = []
bat_avg = []
whip = []
babip = []
lob_pct = []
fip = []
xfip = []
for i, row in batters.iterrows():
    # Pulling stats based on handedness of pitcher
    pitcher_df = pitch_df.sort_values(by = 'IP', ascending = False).loc[(pitch_df['batter']== row['bats'])&(pitch_df['Name']==row['opp_pitch']),:]
    if len(pitcher_df) == 0:
        for col in pitcher_df.columns:
            pitcher_df[col] = [float('NaN')]
    k_9.append(pitcher_df['K/9'].values[0])
    bb_9.append(pitcher_df['BB/9'].values[0])
    k_bb.append(pitcher_df['K/BB'].values[0])
    hr_9.append(pitcher_df['HR/9'].values[0])
    k_pct.append(pitcher_df['K%'].values[0])
    bb_pct.append(pitcher_df['BB%'].values[0])
    k_bb_pct.append(pitcher_df['K-BB%'].values[0])
    bat_avg.append(pitcher_df['AVG'].values[0])
    whip.append(pitcher_df['WHIP'].values[0])
    babip.append(pitcher_df['BABIP'].values[0])
    lob_pct.append(pitcher_df['LOB%'].values[0])
    fip.append(pitcher_df['FIP'].values[0])
    xfip.append(pitcher_df['xFIP'].values[0])
batters.loc[:,'opp_k/9'] = k_9
batters.loc[:,'opp_bb/9'] = k_9
batters.loc[:,'opp_k/bb'] = k_9
batters.loc[:,'opp_hr/9'] = k_9
batters.loc[:,'opp_k_pct'] = k_9
batters.loc[:,'opp_bb_pct'] = k_9
batters.loc[:,'opp_k_bb_pct'] = k_9
batters.loc[:,'opp_avg'] = k_9
batters.loc[:,'opp_whip'] = k_9
batters.loc[:,'opp_babip'] = k_9
batters.loc[:,'opp_lob_pct'] = k_9
batters.loc[:,'opp_fip'] = k_9
batters.loc[:,'opp_xfip'] = k_9

/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [16]:
pitchers = lineups.loc[(lineups[' batting order'] == 'SP')]

In [17]:
pitchers = pitchers[pitchers['Name'].isin(keep_pitch)]

In [18]:
batters

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,opp_pitch_hand,opp_pitch,...,opp_hr/9,opp_k_pct,opp_bb_pct,opp_k_bb_pct,opp_avg,opp_whip,opp_babip,opp_lob_pct,opp_fip,opp_xfip
0,PIT,6/28/2021,1,624428,Adam Frazier,1,N,NaN,L,Kyle Freeland,...,9.346154,9.346154,9.346154,9.346154,9.346154,9.346154,9.346154,9.346154,9.346154,9.346154
1,PIT,6/28/2021,1,663647,Ke'Bryan Hayes,2,N,NaN,L,Kyle Freeland,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PIT,6/28/2021,1,668804,Bryan Reynolds,3,N,NaN,L,Kyle Freeland,...,4.748744,4.748744,4.748744,4.748744,4.748744,4.748744,4.748744,4.748744,4.748744,4.748744
3,PIT,6/28/2021,1,607732,Jacob Stallings,4,N,NaN,L,Kyle Freeland,...,4.748744,4.748744,4.748744,4.748744,4.748744,4.748744,4.748744,4.748744,4.748744,4.748744
4,PIT,6/28/2021,1,595943,Phillip Evans,5,N,NaN,L,Kyle Freeland,...,4.748744,4.748744,4.748744,4.748744,4.748744,4.748744,4.748744,4.748744,4.748744,4.748744
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,LAD,6/28/2021,1,641355,Cody Bellinger,4,N,NaN,R,Anthony DeSclafani,...,7.662162,7.662162,7.662162,7.662162,7.662162,7.662162,7.662162,7.662162,7.662162,7.662162
203,LAD,6/28/2021,1,669257,Will Smith,5,N,NaN,R,Anthony DeSclafani,...,7.235294,7.235294,7.235294,7.235294,7.235294,7.235294,7.235294,7.235294,7.235294,7.235294
204,LAD,6/28/2021,1,621035,Chris Taylor,6,N,NaN,R,Anthony DeSclafani,...,7.235294,7.235294,7.235294,7.235294,7.235294,7.235294,7.235294,7.235294,7.235294,7.235294
205,LAD,6/28/2021,1,666158,Gavin Lux,7,N,NaN,R,Anthony DeSclafani,...,7.662162,7.662162,7.662162,7.662162,7.662162,7.662162,7.662162,7.662162,7.662162,7.662162


In [19]:
num_L_bats = []
num_R_bats = []
for i, row in pitchers.iterrows():
    opp_lineup = batters.loc[(batters['team code'] == matchups[row['team code']]),:]
    find_batters = []
    # There are batters whos data was null due to lack of plate appearances but handedness is known
    for j, row2 in opp_lineup.iterrows():
        if row2['bats'] != 'R' and row2['bats'] != 'L':
            find_bats = bat_df.loc[bat_df['Name'] == row2['Name'],:]
            if len(find_bats) > 0:
                find_batters.append((j, find_bats['bats'].values[0]))
            # If batter handedness is unknown, right is more likely
            if len(find_bats) == 0:
                find_batters.append((j, 'R'))
    for handedness in find_batters:
        opp_lineup.loc[handedness[0], 'bats'] = handedness[1]
    try:
        num_L_bats.append(opp_lineup['bats'].value_counts()['L'])
    except:
        num_L_bats.append(0)
    try:
        num_R_bats.append(opp_lineup['bats'].value_counts()['R'])
    except:
        num_R_bats.append(0)
pitchers['num_L_bats'] = num_L_bats
pitchers['num_R_bats'] = num_R_bats
is_nl = []
for i, row in pitchers.iterrows():
    if row['num_L_bats'] + row['num_R_bats'] == 9:
        is_nl.append(0)
    else:
        is_nl.append(1)
pitchers['is_nl'] = is_nl

/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [20]:
k_9_l = []
bb_9_l = []
k_bb_l = []
hr_9_l = []
k_pct_l = []
bb_pct_l = []
k_bb_pct_l = []
avg_l = []
whip_l = []
babip_l = []
lob_pct_l = []
fip_l = []
xfip_l = []
k_9_r = []
bb_9_r = []
k_bb_r = []
hr_9_r = []
k_pct_r = []
bb_pct_r = []
k_bb_pct_r = []
avg_r = []
whip_r = []
babip_r = []
lob_pct_r = []
fip_r = []
xfip_r = []
# Engineering features
k_9_l_tot = []
bb_9_l_tot = []
k_bb_l_tot = []
hr_9_l_tot = []
k_pct_l_tot = []
bb_pct_l_tot = []
k_bb_pct_l_tot = []
avg_l_tot = []
whip_l_tot = []
babip_l_tot = []
lob_pct_l_tot = []
fip_l_tot = []
xfip_l_tot = []
k_9_r_tot = []
bb_9_r_tot = []
k_bb_r_tot = []
hr_9_r_tot = []
k_pct_r_tot = []
bb_pct_r_tot = []
k_bb_pct_r_tot = []
avg_r_tot = []
whip_r_tot = []
babip_r_tot = []
lob_pct_r_tot = []
fip_r_tot = []
xfip_r_tot = []
for i, row in pitchers.iterrows():
    # Pulling stats of pitcher against lefty hitters
    pitcher_df_l = pitch_df.sort_values(by = 'IP', ascending = False).loc[(pitch_df['batter']== 'L')&(pitch_df['Name']==row['Name']),:]
    if len(pitcher_df_l) == 0:
        for col in pitcher_df_l.columns:
            pitcher_df_l[col] = [float('NaN')]
    k_9_l.append(pitcher_df_l['K/9'].values[0])
    bb_9_l.append(pitcher_df_l['BB/9'].values[0])
    k_bb_l.append(pitcher_df_l['K/BB'].values[0])
    hr_9_l.append(pitcher_df_l['HR/9'].values[0])
    k_pct_l.append(pitcher_df_l['K%'].values[0])
    bb_pct_l.append(pitcher_df_l['BB%'].values[0])
    k_bb_pct_l.append(pitcher_df_l['K-BB%'].values[0])
    avg_l.append(pitcher_df_l['AVG'].values[0])
    whip_l.append(pitcher_df_l['WHIP'].values[0])
    babip_l.append(pitcher_df_l['BABIP'].values[0])
    lob_pct_l.append(pitcher_df_l['LOB%'].values[0])
    fip_l.append(pitcher_df_l['FIP'].values[0])
    xfip_l.append(pitcher_df_l['xFIP'].values[0])
    # Feature Engineering
    k_9_l_tot.append(pitcher_df_l['K/9'].values[0] * row['num_L_bats'])
    bb_9_l_tot.append(pitcher_df_l['BB/9'].values[0] * row['num_L_bats'])
    k_bb_l_tot.append(pitcher_df_l['K/BB'].values[0] * row['num_L_bats'])
    hr_9_l_tot.append(pitcher_df_l['HR/9'].values[0] * row['num_L_bats'])
    k_pct_l_tot.append(pitcher_df_l['K%'].values[0] * row['num_L_bats'])
    bb_pct_l_tot.append(pitcher_df_l['BB%'].values[0] * row['num_L_bats'])
    k_bb_pct_l_tot.append(pitcher_df_l['K-BB%'].values[0] * row['num_L_bats'])
    avg_l_tot.append(pitcher_df_l['AVG'].values[0] * row['num_L_bats'])
    whip_l_tot.append(pitcher_df_l['WHIP'].values[0] * row['num_L_bats'])
    babip_l_tot.append(pitcher_df_l['BABIP'].values[0] * row['num_L_bats'])
    lob_pct_l_tot.append(pitcher_df_l['LOB%'].values[0] * row['num_L_bats'])
    fip_l_tot.append(pitcher_df_l['FIP'].values[0] * row['num_L_bats'])
    xfip_l_tot.append(pitcher_df_l['xFIP'].values[0] * row['num_L_bats'])
    # Pulling stats against righty hitters
    pitcher_df_r = pitch_df.sort_values(by = 'IP', ascending = False).loc[(pitch_df['batter']== 'R')&(pitch_df['Name']== row['Name']),:]
    if len(pitcher_df_r) == 0:
        for col in pitcher_df_r.columns:
            pitcher_df_r[col] = [float('NaN')]
    k_9_r.append(pitcher_df_r['K/9'].values[0])
    bb_9_r.append(pitcher_df_r['BB/9'].values[0])
    k_bb_r.append(pitcher_df_r['K/BB'].values[0])
    hr_9_r.append(pitcher_df_r['HR/9'].values[0])
    k_pct_r.append(pitcher_df_r['K%'].values[0])
    bb_pct_r.append(pitcher_df_r['BB%'].values[0])
    k_bb_pct_r.append(pitcher_df_r['K-BB%'].values[0])
    avg_r.append(pitcher_df_r['AVG'].values[0])
    whip_r.append(pitcher_df_r['WHIP'].values[0])
    babip_r.append(pitcher_df_r['BABIP'].values[0])
    lob_pct_r.append(pitcher_df_r['LOB%'].values[0])
    fip_r.append(pitcher_df_r['FIP'].values[0])
    xfip_r.append(pitcher_df_r['xFIP'].values[0])
    # Feature Engineering
    k_9_r_tot.append(pitcher_df_r['K/9'].values[0] * row['num_R_bats'])
    bb_9_r_tot.append(pitcher_df_r['BB/9'].values[0] * row['num_R_bats'])
    k_bb_r_tot.append(pitcher_df_r['K/BB'].values[0] * row['num_R_bats'])
    hr_9_r_tot.append(pitcher_df_r['HR/9'].values[0] * row['num_R_bats'])
    k_pct_r_tot.append(pitcher_df_r['K%'].values[0] * row['num_R_bats'])
    bb_pct_r_tot.append(pitcher_df_r['BB%'].values[0] * row['num_R_bats'])
    k_bb_pct_r_tot.append(pitcher_df_r['K-BB%'].values[0] * row['num_R_bats'])
    avg_r_tot.append(pitcher_df_r['AVG'].values[0] * row['num_R_bats'])
    whip_r_tot.append(pitcher_df_r['WHIP'].values[0] * row['num_R_bats'])
    babip_r_tot.append(pitcher_df_r['BABIP'].values[0] * row['num_R_bats'])
    lob_pct_r_tot.append(pitcher_df_r['LOB%'].values[0] * row['num_R_bats'])
    fip_r_tot.append(pitcher_df_r['FIP'].values[0] * row['num_R_bats'])
    xfip_r_tot.append(pitcher_df_r['xFIP'].values[0] * row['num_R_bats'])
pitchers['k_9_l'] = k_9_l
pitchers['bb_9_l'] = bb_9_l
pitchers['k_bb_l'] = k_bb_l
pitchers['hr_9_l'] = hr_9_l
pitchers['k_pct_l'] = k_pct_l
pitchers['bb_pct_l'] = bb_pct_l
pitchers['k_bb_pct_l'] = k_bb_pct_l
pitchers['bat_avg_l'] = avg_l
pitchers['whip_l'] = whip_l
pitchers['babip_l'] = babip_l
pitchers['lob_pct_l'] = lob_pct_l
pitchers['fip_l'] = fip_l
pitchers['xfip_l'] = xfip_l
# Right handed batter stats
pitchers['k_9_r'] = k_9_r
pitchers['bb_9_r'] = bb_9_r
pitchers['k_bb_r'] = k_bb_r
pitchers['hr_9_r'] = hr_9_r
pitchers['k_pct_r'] = k_pct_r
pitchers['bb_pct_r'] = bb_pct_r
pitchers['k_bb_pct_r'] = k_bb_pct_r
pitchers['bat_avg_r'] = avg_r
pitchers['whip_r'] = whip_r
pitchers['babip_r'] = babip_r
pitchers['lob_pct_r'] = lob_pct_r
pitchers['fip_r'] = fip_r
pitchers['xfip_r'] = xfip_r
# feature engineered left handed batter stats
pitchers['k_9_l_tot'] = k_9_l_tot
pitchers['bb_9_l_tot'] = bb_9_l_tot
pitchers['k_bb_l_tot'] = k_bb_l_tot
pitchers['hr_9_l_tot'] = hr_9_l_tot
pitchers['k_pct_l_tot'] = k_pct_l_tot
pitchers['bb_pct_l_tot'] = bb_pct_l_tot
pitchers['k_bb_pct_l_tot'] = k_bb_pct_l_tot
pitchers['bat_avg_l_tot'] = avg_l_tot
pitchers['whip_l_tot'] = whip_l_tot
pitchers['babip_l_tot'] = babip_l_tot
pitchers['lob_pct_l_tot'] = lob_pct_l_tot
pitchers['fip_l_tot'] = fip_l_tot
pitchers['xfip_l_tot'] = xfip_l_tot
# feature engineered righ handed batter stats
pitchers['k_9_r_tot'] = k_9_r_tot
pitchers['bb_9_r_tot'] = bb_9_r_tot
pitchers['k_bb_r_tot'] = k_bb_r_tot
pitchers['hr_9_r_tot'] = hr_9_r_tot
pitchers['k_pct_r_tot'] = k_pct_r_tot
pitchers['bb_pct_r_tot'] = bb_pct_r_tot
pitchers['k_bb_pct_r_tot'] = k_bb_pct_r_tot
pitchers['bat_avg_r_tot'] = avg_r_tot
pitchers['whip_r_tot'] = whip_r_tot
pitchers['babip_r_tot'] = babip_r_tot
pitchers['lob_pct_r_tot'] = lob_pct_r_tot
pitchers['fip_r_tot'] = fip_r_tot
pitchers['xfip_r_tot'] = xfip_r_tot

In [21]:
pitchers

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,num_L_bats,num_R_bats,...,hr_9_r_tot,k_pct_r_tot,bb_pct_r_tot,k_bb_pct_r_tot,bat_avg_r_tot,whip_r_tot,babip_r_tot,lob_pct_r_tot,fip_r_tot,xfip_r_tot
8,PIT,6/28/2021,1,542881,Tyler Anderson,SP,N,NaN,3,5,...,6.067416,1.015831,0.448549,0.567282,1.246246,6.573034,1.392157,3.656127,21.827180,25.075016
17,COL,6/28/2021,1,607536,Kyle Freeland,SP,Y,NaN,2,6,...,4.884422,0.739437,0.507042,0.232394,1.724409,8.773869,1.869767,4.924406,27.421427,30.632807
26,PHI,6/28/2021,1,675921,Spencer Howard,SP,Y,NaN,4,4,...,3.600000,1.033708,0.404494,0.629213,1.000000,5.600000,1.283019,3.235294,15.286713,17.085211
35,CIN,6/28/2021,1,489119,Wade Miley,SP,Y,NaN,2,6,...,4.348993,1.100917,0.550459,0.550459,1.500000,8.335570,1.764706,4.285714,23.116537,23.082316
45,LAA,6/28/2021,1,605164,Dylan Bundy,SP,N,NaN,1,8,...,8.894118,2.344828,0.413793,1.931034,1.794393,8.470588,2.309859,4.812030,25.315996,25.850747
73,WAS,6/28/2021,1,502179,Paolo Espino,SP,N,NaN,5,3,...,4.500000,0.847826,0.130435,0.717391,0.545455,2.500000,0.620690,2.916667,10.962091,10.754179
83,KC,6/28/2021,1,518633,Danny Duffy,SP,N,NaN,1,8,...,8.000000,1.953488,0.720930,1.232558,1.960784,10.469136,2.422018,6.037736,30.096618,37.661676
93,BOS,6/28/2021,1,572070,Garrett Richards,SP,Y,NaN,3,6,...,3.620112,1.308642,0.641975,0.666667,1.273585,7.139665,1.566879,4.736842,22.288288,28.262609
113,CLE,6/28/2021,1,669212,Eli Morgan,SP,N,NaN,5,4,...,0.000000,0.266667,0.533333,-0.266667,2.000000,12.000000,2.000000,2.000000,18.550317,29.023462
122,CHC,6/28/2021,1,543294,Kyle Hendricks,SP,N,NaN,3,5,...,7.264574,0.770492,0.147541,0.622951,1.335616,5.852018,1.404255,3.947368,22.214443,20.215963


**Data Description:** This worksheet provides a framework for creating dataframes for each days' players and stats. I will combine these dataframes across many days to get my final training data. These data frames have all of the stats that I could potentially be interested in, the features will be reduced for modeling as there is a significant amount of correlation between many of the features. The batters dataframe includes the batters stats against pitchers who throw with the same hand as the opposing pitcher, and includes the stats of how the opposing pitcher performs against batters with the same handedness of the batter. The pitchers dataframe contains each pitcher and their stats against both right handed and left handed batters. It also contains the number of right and left handed batters in the opposing lineup. I engineered features that are the pitchers stats vs the handedness of a batter multiplied by the number of those handed batters in the lineup. My final feature is is_nl. This is 1 if the lineup they are facing includes a pitcher and 0 if not.

**Data Dictionary:**   
team code: The team the player plays for.  
Name: The players name  
Batting order: The order the batters hit in the lineup. Is SP if player is a pitcher.  
opp_pitch_hand(batters): The hand that the opposing pitcher throws with  
opp_pitch(batters):The name of the opposing pitcher. 
opp_x(batters): This is the format for column names of stats that the opposing pitcher has against batters of the same handedness of the batter.  
(batters): There are also many columns of stats for how the batter performs against pitchers with the same handedness of the opposing pitcher.    
num_L_bats(pitchers): The number of left handed batters in the opposing lineup.  
num_R_bats(pitchers): The number of right handed batters in the opposing lineup.  
x_(l/r) (pitchers): stats describing how the pitcher performs against either left or right handed batters.    
x_(l/r)_tot (pitchers): The stats of how a pitcher performs against both hands multiplied by the number of those handed batters in the lineup.

In [22]:
scores = pd.read_csv('../datasets/6_28_target.csv', delimiter = ';')
scores.head()

,Date,GID,MLB_ID,Name,Starter,Bat order,DK posn,DK pts,DK sal,Team,Oppt,dblhdr,Tm Runs,Opp Runs
0,20210628,140b,607536,"Freeland, Kyle",1.0,9,1.0,26.25,6400.0,COL,v pit,NaN,2,0
1,20210628,109z,545333,"Bauer, Trevor",1.0,9,1.0,24.10,10900.0,LAD,v sfo,NaN,3,2
2,20210628,153v,660761,"Suarez, Jose",NaN,0,1.0,22.20,8600.0,LAA,@ nyy,NaN,5,3
3,20210628,147h,642547,"Peralta, Freddy",1.0,9,1.0,21.90,10500.0,MIL,v chc,NaN,14,4
4,20210628,141f,502179,"Espino, Paolo",1.0,9,1.0,18.25,5600.0,WAS,v nym,NaN,8,4


In [23]:
def clean_names(name):
    name = name.replace(',','')
    name = name.replace('.','')
    name = name.replace('-hitter','')
    name = name.replace('-pitcher','')
    names = name.split()
    first = names.pop(-1)
    names.insert(0, first)
    name = ' '.join(names)
    name = name.replace('James Happ', 'JA Happ')
    return name

In [24]:
scores['Name'] = [clean_names(name) for name in scores['Name']]

In [25]:
pitch_s = scores[scores['DK posn'] == 1]
hit_s = scores[scores['DK posn'] != 1]

In [26]:
pitch_final = pd.merge(left = pitchers, right = pitch_s, how = 'inner')

In [27]:
bat_final = pd.merge(left = batters, right = hit_s,
         how = 'inner').dropna(how = 'any', subset = ['bats', 'DK pts', 'Starter'])

In [28]:
bat_final = bat_final[bat_final['dblhdr'].isna()]

In [29]:
bat_final.to_csv('../merged_data/hitters_6_28.csv')
pitch_final.to_csv('../merged_data/pitchers_6_28.csv')

In [30]:
bat_final

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,opp_pitch_hand,opp_pitch,...,Starter,Bat order,DK posn,DK pts,DK sal,Team,Oppt,dblhdr,Tm Runs,Opp Runs
2,PIT,6/28/2021,1,668804,Bryan Reynolds,3,N,NaN,L,Kyle Freeland,...,1.0,3,7.0,8.0,3900.0,PIT,@ col,NaN,0,2
3,PIT,6/28/2021,1,607732,Jacob Stallings,4,N,NaN,L,Kyle Freeland,...,1.0,5,2.0,6.0,3500.0,PIT,@ col,NaN,0,2
4,PIT,6/28/2021,1,595943,Phillip Evans,5,N,NaN,L,Kyle Freeland,...,1.0,6,37.0,0.0,3000.0,PIT,@ col,NaN,0,2
5,PIT,6/28/2021,1,570481,Erik Gonzalez,6,N,NaN,L,Kyle Freeland,...,1.0,1,36.0,3.0,2700.0,PIT,@ col,NaN,0,2
6,PIT,6/28/2021,1,621028,Kevin Newman,8,N,NaN,L,Kyle Freeland,...,1.0,8,6.0,6.0,2900.0,PIT,@ col,NaN,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,LAD,6/28/2021,1,641355,Cody Bellinger,4,N,NaN,R,Anthony DeSclafani,...,1.0,4,7.0,0.0,3900.0,LAD,v sfo,NaN,3,2
105,LAD,6/28/2021,1,669257,Will Smith,5,N,NaN,R,Anthony DeSclafani,...,1.0,5,2.0,14.0,4100.0,LAD,v sfo,NaN,3,2
106,LAD,6/28/2021,1,621035,Chris Taylor,6,N,NaN,R,Anthony DeSclafani,...,1.0,6,47.0,0.0,4500.0,LAD,v sfo,NaN,3,2
107,LAD,6/28/2021,1,666158,Gavin Lux,7,N,NaN,R,Anthony DeSclafani,...,1.0,7,6.0,3.0,3700.0,LAD,v sfo,NaN,3,2


In [31]:
pitch_final

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,num_L_bats,num_R_bats,...,Starter,Bat order,DK posn,DK pts,DK sal,Team,Oppt,dblhdr,Tm Runs,Opp Runs
0,PIT,6/28/2021,1,542881,Tyler Anderson,SP,N,NaN,3,5,...,1.0,9,1.0,9.65,7700.0,PIT,@ col,NaN,0,2
1,COL,6/28/2021,1,607536,Kyle Freeland,SP,Y,NaN,2,6,...,1.0,9,1.0,26.25,6400.0,COL,v pit,NaN,2,0
2,PHI,6/28/2021,1,675921,Spencer Howard,SP,Y,NaN,4,4,...,1.0,9,1.0,0.25,6900.0,PHI,@ cin,NaN,4,12
3,CIN,6/28/2021,1,489119,Wade Miley,SP,Y,NaN,2,6,...,1.0,9,1.0,12.90,9000.0,CIN,v phi,NaN,12,4
4,LAA,6/28/2021,1,605164,Dylan Bundy,SP,N,NaN,1,8,...,1.0,0,1.0,-2.65,8200.0,LAA,@ nyy,NaN,5,3
5,WAS,6/28/2021,1,502179,Paolo Espino,SP,N,NaN,5,3,...,1.0,9,1.0,18.25,5600.0,WAS,v nym,NaN,8,4
6,KC,6/28/2021,1,518633,Danny Duffy,SP,N,NaN,1,8,...,1.0,0,1.0,-0.10,8800.0,KAN,@ bos,NaN,5,6
7,BOS,6/28/2021,1,572070,Garrett Richards,SP,Y,NaN,3,6,...,1.0,0,1.0,2.15,6900.0,BOS,v kan,NaN,6,5
8,CLE,6/28/2021,1,669212,Eli Morgan,SP,N,NaN,5,4,...,1.0,0,1.0,11.65,6200.0,CLE,v det,NaN,13,5
9,CHC,6/28/2021,1,543294,Kyle Hendricks,SP,N,NaN,3,5,...,1.0,9,1.0,8.10,9100.0,CHC,@ mil,NaN,4,14
